In [1]:
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import jaccard_score, multilabel_confusion_matrix, precision_recall_fscore_support, precision_score
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from skopt import BayesSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.svm import SVC

In [2]:
x = np.load('../Part1.npy')
Y_t = np.load('../train.npy/y_train.npy')
#X_train = (x-np.min(x))/(np.max(x)-np.min(x))
xt = np.load('../Part2.npy')
Y_test = np.load('../test.npy/Y_Test.npy')
#X_test = (xt-np.min(xt))/(np.max(xt)-np.min(xt))
x_add = np.load('../Part3.npy')
y_add = np.load('../additional.npy/Y_additional.npy')
#x_add = (x_add-np.min(x_add))/(np.max(x_add)-np.min(x_add))
Y_train = np.concatenate((Y_t, Y_test, y_add))
X_train = np.concatenate((x, xt, x_add))
#X_train = (X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train))
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [3]:
X_test = np.load('X_single.npy')
Y_test = np.load('Y_single.npy')
scaler = preprocessing.StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [7]:
class OvrMLP(BaseEstimator, ClassifierMixin):
    def __init__(self, hidden_layer_sizes, activation, solver, alpha, batch_size, learning_rate, learning_rate_init, 
                 max_iter, shuffle, warm_start):
        super(OvrMLP, self).__init__()
        self.activation = activation
        self.hidden_layer_sizes = hidden_layer_sizes
        self.alpha = alpha
        self.learning_rate_init = learning_rate_init
        self.max_iter = max_iter
        self.model = MLPClassifier(activation = activation, hidden_layer_sizes = hidden_layer_sizes ,solver = 'adam',
                                   alpha = alpha, batch_size = 200, learning_rate_init= learning_rate_init, max_iter = max_iter)
        self.ovr = OneVsRestClassifier(self.model)
        self.solver = solver
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.shuffle = shuffle
        self.warm_start = warm_start
    def fit(self, X_train, Y_train):
        #self.chain = ClassifierChain(self.model, order="random", random_state=1)
        return self.ovr.fit(X_train, Y_train)
    
    def score(self, X_test, Y_test):
        Y_pred_chain = self.ovr.predict(X_test)
        chain_jaccard_score = jaccard_score(Y_test, Y_pred_chain, average="samples")
        return chain_jaccard_score
        

In [8]:
search_space = {
    'hidden_layer_sizes': [(100), (200), (50)],
    'activation': ['identity', 'logistic', 'relu', 'tanh'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': (1e-5, 1e-2, 'log-uniform'),
    'batch_size': [50, 100, 200, 500],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': (0.0001, 0.1, 'log-uniform'),
    'max_iter': (50, 200),
    'shuffle': [True, False],
    'warm_start':[True, False],
    
}

test = OvrMLP((100), activation = 'tanh', solver = 'adam', alpha= 0.001,learning_rate='constant', 
                learning_rate_init= 0.01, batch_size=200, max_iter = 200, shuffle = True, warm_start = True)
opt = BayesSearchCV(test, search_space, n_iter=50, cv=7, n_jobs=-1)
opt.fit(X_train, Y_train)

C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (193) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (193) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (193) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (193) reached and the optimization hasn't converged yet.
  war

BayesSearchCV(cv=7,
              estimator=OvrMLP(activation='tanh', alpha=0.001, batch_size=200,
                               hidden_layer_sizes=100, learning_rate='constant',
                               learning_rate_init=0.01, max_iter=200,
                               shuffle=True, solver='adam', warm_start=True),
              n_jobs=-1,
              search_spaces={'activation': ['identity', 'logistic', 'relu',
                                            'tanh'],
                             'alpha': (1e-05, 0.01, 'log-uniform'),
                             'batch_size': [50, 100, 200, 500],
                             'hidden_layer_sizes': [100, 200, 50],
                             'learning_rate': ['constant', 'invscaling',
                                               'adaptive'],
                             'learning_rate_init': (0.0001, 0.1, 'log-uniform'),
                             'max_iter': (50, 200), 'shuffle': [True, False],
                             'solver': ['lbfgs', 'sgd', 'adam'],
                             'warm_start': [True, False]})

In [9]:
print("Best parameters found: ", opt.best_params_)
print("Best score: ", opt.best_score_)

Best parameters found:  OrderedDict([('activation', 'logistic'), ('alpha', 0.005319677452419251), ('batch_size', 200), ('hidden_layer_sizes', 100), ('learning_rate', 'constant'), ('learning_rate_init', 0.0041260189221123044), ('max_iter', 193), ('shuffle', True), ('solver', 'lbfgs'), ('warm_start', True)])
Best score:  0.19033546122739642


In [11]:
import warnings
warnings.filterwarnings("ignore")
mlp = MLPClassifier(activation = 'logistic',hidden_layer_sizes = [100] ,solver = 'lbfgs',alpha = 0.0053, batch_size = 200, 
                    learning_rate='constant', learning_rate_init=0.0041, max_iter = 193, warm_start=True, shuffle=True)
ovr = OneVsRestClassifier(mlp)
ovr.fit(X_train, Y_train)

OneVsRestClassifier(estimator=MLPClassifier(activation='logistic', alpha=0.0053,
                                            batch_size=200,
                                            hidden_layer_sizes=[100],
                                            learning_rate_init=0.0041,
                                            max_iter=193, solver='lbfgs',
                                            warm_start=True))

In [12]:
pred = ovr.predict(X_test)
m = multilabel_confusion_matrix(Y_test, pred)
print(m)

[[[6148  169]
  [ 371   17]]

 [[6139   61]
  [ 492   13]]

 [[6045  209]
  [ 439   12]]

 [[4946 1122]
  [ 465  172]]

 [[3920 2025]
  [ 449  311]]

 [[5285  738]
  [ 582  100]]

 [[4215 1769]
  [ 483  238]]

 [[5659  420]
  [ 570   56]]

 [[5918  210]
  [ 554   23]]

 [[5802  323]
  [ 540   40]]

 [[3414 2513]
  [ 419  359]]]


In [14]:
m[0]

array([[6148,  169],
       [ 371,   17]], dtype=int64)